# Rolling balls tutorial

## The modelling problem

Suppose there's a ball rolling along a pool table. It's rolling directly towards a pocket but we're not sure it's got enough momentum to make it all the way.

Suppose we've rigged up a camera measuring the position of a ball as it rolls. To make it easy, we've already got some image processing code to calculate the distance from the centre of the ball to the edge of the pocket. Our aim in this tutorial is to use the camera measurements to work out whether the ball will make it to the pocket.

The camera's frame rate is 50 frames/sec and we receive the following distance measurements (in meters) from consecutive frames:

In [5]:
data = [
    0.6180,
    0.6111,
    0.6042,
    0.5973,
    0.5905,
    0.5837,
    0.5770,
    0.5703]

The standard error of the measurements is ±0.5mm. 

## Designing a model

Let's begin by thinking about the mechanisms that may effect the motion of the ball. Here are a few:

* the friction between the ball and the table as it rolls
* air resistance on the ball
* if the ball has backspin: the friction of the ball as it skids along the table

The study by <a href="WittersDuymelinck86.pdf">Witters and Duymelinck (1986)</a> shows that we can neglect air resistance at the speeds we're concered about. It also shows that the ball is unlikely to have any backspin at this speed, so we make the modelling assumption that the ball is rolling along the table with no backspin. So, we are left with just the friction between the ball and the table, which the paper shows exerts a decelleration force that is proportional to the force of the ball on the table due to gravity. If we let $\alpha$ be the constant of proportionality then the frictional force $F = \alpha mg$.

### First attempt: Forward integration model

Using <a href="https://en.wikipedia.org/wiki/Newton%27s_laws_of_motion">Newton's second law of motion</a>, namely $\vec{F}=m\vec{a}$, we can fomulate an equation for the acceleration of the ball given its mass and the frictional force: 
$$
a = \frac{d^2s}{dt^2} = \alpha g
$$
Using this, we can easily write a program to simulate the position and velocity of the ball, given the frictional coefficient $\alpha$ and an initial velocity:

In [6]:
# s is initial position
# v is initial velocity
# alpha is the coefficient of friction
# returns the final resting position of the ball
def restingPosition(s, v, alpha):
    g = 9.8 # gravitational acceleration
    a = alpha*g # acceleration
    dt = 0.001 # time step

    while(v < 0):
      s = s + v*dt;
      v = v + a*dt;
    return s;


This is OK, but to calculate the final resting place, we need to know the initial velocity, initial position and frictional coefficient. However, the data doesn't directly tell us any of these things so we need to do some extra maths to work out these numbers from the data.

As a first approximation, we could take the inital position to be the first measured frame, the initial velocity to be the average velocity between the first two frames and the acceleration to be the change of velocity between the first two  and last two frames:

In [7]:
frame_period = 1.0/50.0
g = 9.8
s = data[0]
v0 = (data[1] - data[0])/frame_period
v1 = (data[7] - data[6])/frame_period
alpha = (v1-v0)/(6*frame_period*g)

print "Initial position = ", s
print "Initial velocity = ", v0
print "alpha = ", alpha
print "Final resting distance = ", restingPosition(s,v0,alpha)

Initial position =  0.618
Initial velocity =  -0.345
alpha =  0.00850340136055
Final resting distance =  -0.0963224999996


The final resting distance should be negative, meaning that the ball makes it past the edge of the pocket and is potted.

This is OK, but it isn't making use of the third, fourth and fifth data points and we don't know how much uncertainty there is in our final result. To do better we need to abstract a little bit further.

### Second attempt: Maximum likelihood

Going back to the equation of motion, notice that we can integrate this to give the velocity of the ball
$$
v = \frac{ds}{dt} = u + \alpha gt
$$
and integrating this again gives the position at any given time
$$
s = s_0 + ut + \frac{1}{2}\alpha gt^2
$$

Using this equation as our equation of motion allows us to do much more.

If we let $Y=[d_0...d_7]$ be the data from the camera, then we can link the data to the model as a set of eight simultaneous equations of the form
$$
d_n + \epsilon_n = s_0 + u\left(\frac{n}{50}\right) + \frac{1}{2}\alpha g \left(\frac{n}{50}\right)^2
$$
for $n = 0...7$, where $\epsilon_n$ is the measurement error for the $n^{th}$ frame.

This can be expressed in matrix notation by letting
$$
B = \left[\begin{array}{c}
s_0 \\ u \\ \alpha
\end{array}\right]
$$ 
$$
X = \left[ 
\begin{array}{ccc}
1 & \frac{1}{50} & \frac{1}{2}g\left(\frac{1}{50}\right)^2\\
1 & \frac{2}{50} & \frac{1}{2}g\left(\frac{2}{50}\right)^2\\
& \vdots & \\
1 & \frac{7}{50} & \frac{1}{2}g\left(\frac{7}{50}\right)^2\\
\end{array}
\right]
$$
and
$$
E = \left[\begin{array}{c}
\epsilon_0 \\ \vdots \\ \epsilon_7
\end{array}\right]
$$
giving
$$
Y + E = XB
$$
The only unknowns in this equation are $E$, the measurement error terms, and $X$, the parameters we need for our model.

A commom thing to do at this point is to run a <a href="https://en.wikipedia.org/wiki/Linear_regression">linear regression</a> on the above equation. This finds the value of $X$ that minimises $E^TE$ (i.e. the $X$ that maximises the likelihood of getting the measurements we actually observed).

We can do this in Python using the `numpy` function `linalg.lstsq`, as follows:

In [57]:
import numpy as np

t = np.array([0, 1.0/50, 2.0/50, 3.0/50, 4.0/50, 5.0/50, 6.0/50, 7.0/50])

X = np.vstack([np.ones(len(t)), t, 0.5*g*t*t]).T
Y = np.array(data).T
B = np.linalg.lstsq(X, Y)[0]

print "B = ", B
print "D = ", D
print "X = ", X

B =  [[ 1.       0.       0.     ]
 [ 1.       0.02     0.00196]
 [ 1.       0.04     0.00784]
 [ 1.       0.06     0.01764]
 [ 1.       0.08     0.03136]
 [ 1.       0.1      0.049  ]
 [ 1.       0.12     0.07056]
 [ 1.       0.14     0.09604]]
D =  [ 0.618   0.6111  0.6042  0.5973  0.5905  0.5837  0.577   0.5703]
X =  [ 0.61802083 -0.34818452  0.01062925]


The regressed value of $X$ gives us the initial position, velocity and coefficient of friction. With this we can find the resting position of the ball. First, setting the equation for velocity to zero, we find the time at which the ball comes to rest is given by

$$
t_r = \frac{-u}{\alpha g}
$$

substituting this into the equation for position and simplifying gives

$$
s_r = s_0 - \frac{u^2}{2\alpha g}
$$

We can now calculate whether the ball is potted in the maximum likelihood scenario: 

In [74]:
s0 = 0.61802083
u = -0.34818452
alpha = 0.01062925
g= 9.8

sr = s0 - u*u/(2*alpha*g)
print sr

0.0361049290488


With this new analysis, the ball stops 36mm short of the pocket!

This is a much better model than our first attempt, but even this one is a bit simplistic. The problem is that 